In [2]:
import numpy as np
import matplotlib.pyplot as plt

def load_sensor_data(filename):
    """
    np.savez로 저장된 .npz 파일을 메모리 맵 모드로 불러옵니다.
    """
    print(f"Loading data from {filename}...")
    
    try:
        # mmap_mode='r'로 변경하여 로딩 속도 향상 (압축 안 했을 때 효과적)
        data = np.load(filename, mmap_mode='r') 
        
        timestamps = data['timestamps']
        send_timestamps = data['send_timestamps']
        forces = data['forces']
        alines = data['alines']
        
        print("✅ 데이터 로드 성공!")
        print("-" * 30)
        print(f"  timestamps: {timestamps.shape}")
        print(f"  send_timestamps: {send_timestamps.shape}")
        print(f"  forces: {forces.shape}")
        print(f"  alines: {alines.shape}")
        print("-" * 30)
        
        return {
            "timestamps": timestamps,
            "send_timestamps": send_timestamps,
            "forces": forces,
            "alines": alines
        }

    except FileNotFoundError:
        print(f"[ERROR] 파일을 찾을 수 없습니다: {filename}")
    except Exception as e:
        print(f"[ERROR] 데이터 로드 실패: {e}")
    
    return None

# ▼▼▼ [수정] Matplotlib 시각화 함수 (2개 플롯으로 분리) ▼▼▼
def visualize_separate_plots(data, base_filename="sensor_plot"):
    """
    로드된 데이터를 matplotlib을 사용해 별도의 두 개 플롯으로 시각화합니다.
    1. [base_filename]_force_timing.png: Force, C++ 큐 지연, 측정 간격
    2. [base_filename]_oct_alines.png: A-lines (2D 이미지)
    """
    if not data:
        print("시각화할 데이터가 없습니다.")
        return

    timestamps = data['timestamps']
    send_timestamps = data['send_timestamps']
    forces = data['forces']
    alines = data['alines'] # Shape: (N, 1025)

    if len(timestamps) < 2:
        print("데이터가 너무 적어 간격 계산 및 시각화가 어렵습니다.")
        return

    # 시간 축 계산 (0초부터 시작)
    time_elapsed = timestamps - timestamps[0]

    # 1. 측정 간격 계산 (ms 단위)
    intervals = np.diff(timestamps) * 1000 
    
    # 2. C++ 큐 지연 계산 (ms 단위)
    cpp_queue_delay = (send_timestamps - timestamps) * 1000

    # --- 플롯 1: Force 및 타이밍 (측정 간격, C++ 지연) ---
    print(f"Generating Force/Timing plot...")
    # 2행 1열, x축 공유, 비율 1:2
    fig1, axes1 = plt.subplots(2, 1, figsize=(15, 8), sharex=True, 
                             gridspec_kw={'height_ratios': [1, 2]}) 
    ax0, ax1 = axes1

    # 1-0. 측정 간격 그래프 (ax0)
    ax0.plot(time_elapsed[1:], intervals, label='Sampling Interval (ms)', color='g', alpha=0.7)
    ax0.set_ylabel('Interval (ms)')
    ax0.set_title('Sensor Data Analysis: Timing and Force')
    ax0.grid(True)
    ax0.legend()

    # 1-1. Force 그래프 (ax1) + C++ 큐 지연 (보조 축)
    ax1.plot(time_elapsed, forces, label='Force', color='b')
    ax1.set_ylabel('Force', color='b')
    ax1.tick_params(axis='y', labelcolor='b')
    ax1.grid(True)
    
    ax1_twin = ax1.twinx() 
    ax1_twin.plot(time_elapsed, cpp_queue_delay, label='C++ Queue Delay (ms)', color='r', linestyle='--', alpha=0.6)
    ax1_twin.set_ylabel('C++ Queue Delay (ms)', color='r')
    ax1_twin.tick_params(axis='y', labelcolor='r')
    
    lines, labels = ax1.get_legend_handles_labels()
    lines2, labels2 = ax1_twin.get_legend_handles_labels()
    ax1.legend(lines + lines2, labels + labels2, loc='upper left')
    
    ax1.set_xlabel('Time (seconds)') # X축 라벨을 아래쪽 그래프에만 추가
    
    plt.tight_layout()
    force_plot_filename = f"{base_filename}_force_timing.png"
    plt.savefig(force_plot_filename) # 파일로 저장
    plt.close(fig1) # Figure 닫기
    print(f"✅ Force/Timing plot saved to {force_plot_filename}")

    # --- 플롯 2: A-line (OCT) 이미지 ---
    print(f"Generating OCT A-lines plot...")
    fig2, ax2 = plt.subplots(1, 1, figsize=(15, 8)) # 1x1
    
    im = ax2.imshow(alines.T, 
                    cmap='gray', 
                    aspect='auto', 
                    interpolation='nearest',
                    origin='lower',
                    extent=[time_elapsed[0], time_elapsed[-1], 0, alines.shape[1]])
    
    ax2.set_xlabel('Time (seconds)')
    ax2.set_ylabel('A-line Index (0-1024)')
    ax2.set_title('A-lines (OCT Data)')
    # fig2.colorbar(im, ax=ax2, label='Intensity', shrink=0.8) # 컬러바 필요시 주석 해제
    
    plt.tight_layout()
    oct_plot_filename = f"{base_filename}_oct_alines.png"
    plt.savefig(oct_plot_filename) # 파일로 저장
    plt.close(fig2) # Figure 닫기
    print(f"✅ OCT A-lines plot saved to {oct_plot_filename}")
# ▲▲▲ [수정] ▲▲▲

if __name__ == "__main__":
    # 불러올 파일명을 지정합니다.
    filename_to_load = "../Real_Env_Test/recv_all_20251023_174822/sensor_data_20251023_174822_1761209343.npz" 
    
    loaded_data = load_sensor_data(filename_to_load)
    
    if loaded_data:
        # [수정] 분리된 플롯 생성 함수 호출
        # 출력 파일명의 접두사(base_filename)를 지정할 수 있습니다.
        visualize_separate_plots(loaded_data, base_filename="my_sensor_data_plot")

Loading data from ../Real_Env_Test/recv_all_20251023_174822/sensor_data_20251023_174822_1761209343.npz...
✅ 데이터 로드 성공!
------------------------------
  timestamps: (105372,)
  send_timestamps: (105372,)
  forces: (105372,)
  alines: (105372, 1025)
------------------------------
Generating Force/Timing plot...
✅ Force/Timing plot saved to my_sensor_data_plot_force_timing.png
Generating OCT A-lines plot...
✅ OCT A-lines plot saved to my_sensor_data_plot_oct_alines.png


In [5]:
import numpy as np
import matplotlib.pyplot as plt

def load_sensor_data(filename):
    """
    np.savez로 저장된 .npz 파일을 메모리 맵 모드로 불러옵니다.
    """
    print(f"Loading data from {filename}...")
    
    try:
        # mmap_mode='r'로 변경하여 로딩 속도 향상 (압축 안 했을 때 효과적)
        data = np.load(filename, mmap_mode='r') 
        
        timestamps = data['timestamps']
        # send_timestamps는 이제 시각화에 사용되지 않습니다.
        # send_timestamps = data['send_timestamps'] 
        forces = data['forces']
        alines = data['alines']
        
        print("✅ 데이터 로드 성공!")
        print("-" * 30)
        print(f"  timestamps: {timestamps.shape}")
        # print(f"  send_timestamps: {send_timestamps.shape}")
        print(f"  forces: {forces.shape}")
        print(f"  alines: {alines.shape}")
        print("-" * 30)
        
        return {
            "timestamps": timestamps,
            # "send_timestamps": send_timestamps, # 필요 없음
            "forces": forces,
            "alines": alines
        }

    except FileNotFoundError:
        print(f"[ERROR] 파일을 찾을 수 없습니다: {filename}")
    except Exception as e:
        print(f"[ERROR] 데이터 로드 실패: {e}")
    
    return None

# ▼▼▼ [수정] Matplotlib 시각화 함수 (2개 플롯으로 분리, Delay 제거) ▼▼▼
def visualize_separate_plots(data, base_filename="sensor_plot"):
    """
    로드된 데이터를 matplotlib을 사용해 별도의 두 개 플롯으로 시각화합니다.
    (Delay 그래프 제외)
    
    1. [base_filename]_force_timing.png: 측정 간격, Force
    2. [base_filename]_oct_alines.png: A-lines (2D 이미지)
    """
    if not data:
        print("시각화할 데이터가 없습니다.")
        return

    timestamps = data['timestamps']
    forces = data['forces']
    alines = data['alines'] # Shape: (N, 1025)

    if len(timestamps) < 2:
        print("데이터가 너무 적어 간격 계산 및 시각화가 어렵습니다.")
        return

    # 시간 축 계산 (0초부터 시작)
    time_elapsed = timestamps - timestamps[0]

    # 1. 측정 간격 계산 (ms 단위)
    intervals = np.diff(timestamps) * 1000 
    
    # --- 플롯 1: Force 및 타이밍 (측정 간격) ---
    print(f"Generating Force/Timing plot...")
    # 2행 1열, x축 공유, 비율 1:2
    fig1, axes1 = plt.subplots(2, 1, figsize=(15, 8), sharex=True, 
                             gridspec_kw={'height_ratios': [1, 2]}) 
    ax0, ax1 = axes1

    # 1-0. 측정 간격 그래프 (ax0)
    # np.diff는 길이가 N-1 이므로 x축도 맞춰줌
    ax0.plot(time_elapsed[1:], intervals, label='Sampling Interval (ms)', color='g', alpha=0.7)
    ax0.set_ylabel('Interval (ms)')
    ax0.set_title('Sensor Data Analysis: Timing and Force')
    ax0.grid(True)
    ax0.legend()

    # 1-1. Force 그래프 (ax1)
    ax1.plot(time_elapsed, forces, label='Force', color='b')
    ax1.set_ylabel('Force', color='b')
    ax1.tick_params(axis='y', labelcolor='b')
    ax1.grid(True)
    ax1.legend(loc='upper left') # Delay가 없으므로 간단히 처리
    
    ax1.set_xlabel('Time (seconds)') # X축 라벨을 아래쪽 그래프에만 추가
    
    plt.tight_layout()
    force_plot_filename = f"{base_filename}_force_timing.png"
    plt.savefig(force_plot_filename) # 파일로 저장
    plt.close(fig1) # Figure 닫기
    print(f"✅ Force/Timing plot saved to {force_plot_filename}")

    # --- 플롯 2: A-line (OCT) 이미지 ---
    print(f"Generating OCT A-lines plot...")
    fig2, ax2 = plt.subplots(1, 1, figsize=(15, 8)) # 1x1
    
    im = ax2.imshow(alines.T, 
                    cmap='gray', 
                    aspect='auto', 
                    interpolation='nearest',
                    origin='lower',
                    extent=[time_elapsed[0], time_elapsed[-1], 0, alines.shape[1]])
    
    ax2.set_xlabel('Time (seconds)')
    ax2.set_ylabel('A-line Index (0-1024)')
    ax2.set_title('A-lines (OCT Data)')
    # fig2.colorbar(im, ax=ax2, label='Intensity', shrink=0.8) # 컬러바 필요시 주석 해제
    
    plt.tight_layout()
    oct_plot_filename = f"{base_filename}_oct_alines.png"
    plt.savefig(oct_plot_filename) # 파일로 저장
    plt.close(fig2) # Figure 닫기
    print(f"✅ OCT A-lines plot saved to {oct_plot_filename}")
# ▲▲▲ [수정] ▲▲▲

if __name__ == "__main__":
    # 불러올 파일명을 지정합니다.
    filename_to_load = "../Real_Env_Test/recv_all_20251023_175947/sensor_data_20251023_175947_1761210017.npz" 
    
    loaded_data = load_sensor_data(filename_to_load)
    
    if loaded_data:
        # [수정] 분리된 플롯 생성 함수 호출
        visualize_separate_plots(loaded_data, base_filename="my_sensor_data_plot")

Loading data from ../Real_Env_Test/recv_all_20251023_175947/sensor_data_20251023_175947_1761210017.npz...
✅ 데이터 로드 성공!
------------------------------
  timestamps: (46663,)
  forces: (46663,)
  alines: (46663, 1025)
------------------------------
Generating Force/Timing plot...


✅ Force/Timing plot saved to my_sensor_data_plot_force_timing.png
Generating OCT A-lines plot...
✅ OCT A-lines plot saved to my_sensor_data_plot_oct_alines.png
